In [1]:
import snowflake.connector

In [2]:
conn = snowflake.connector.connect(
    user='second2',
    password='gyczeg6kaHqywownor',
    account='qbmhuza-bnb86629'
)


In [3]:
cs = conn.cursor()

In [4]:
cs.execute("CREATE WAREHOUSE IF NOT EXISTS my_first_warehouse")

In [5]:
cs.execute("CREATE DATABASE IF NOT EXISTS testdb")

In [11]:
from pathlib import Path
import re

# ---- 0) connect  ----
conn = snowflake.connector.connect(
    host="qbmhuza-bnb86629.snowflakecomputing.com",   
    account="qbmhuza-bnb86629",                        
    user="second2",
    password="gyczeg6kaHqywownor",
    warehouse="COMPUTE_WH",                           
)
cs = conn.cursor()

# make sure the warehouse will wake itself
cs.execute("ALTER WAREHOUSE COMPUTE_WH SET AUTO_SUSPEND=60 AUTO_RESUME=TRUE")

# ---- 1) set DB/Schema (use your existing TESTDB) ----
DB, SCHEMA = "TESTDB", "PUBLIC"
cs.execute(f"CREATE DATABASE IF NOT EXISTS {DB}")
cs.execute(f"CREATE SCHEMA   IF NOT EXISTS {DB}.{SCHEMA}")
cs.execute(f"USE DATABASE {DB}")
cs.execute(f"USE SCHEMA {SCHEMA}")

# ---- 2) read and run your .pgsql file from the repo ----
sql_path = Path("/home/jovyan/Desktop/MSBA/SQL & ETL /MGTA_464_Snowflake_Project/Data/supplier_case.pgsql")     
assert sql_path.exists(), f"Not found: {sql_path.resolve()}"
txt = sql_path.read_text(encoding="utf-8")

# tiny Postgres -> Snowflake cleanups
txt = "\n".join(l for l in txt.splitlines() if not l.strip().startswith("\\"))            # drop psql meta commands
txt = re.sub(r"\bNUMERIC\b", "NUMBER", txt, flags=re.I)                                   # NUMERIC -> NUMBER (safe)
txt = re.sub(r"\bsupplier_case\b", f"{DB}.{SCHEMA}.SUPPLIER_CASE", txt, flags=re.I)       # fully-qualify table

# split on semicolons and execute
stmts = [s.strip() for s in re.split(r";\s*(?=\n|$)", txt) if s.strip()]
for s in stmts:
    cs.execute(s)

# ---- 3) visualize (still only cs.execute) ----
print("Rows:", cs.execute(f"SELECT COUNT(*) FROM {DB}.{SCHEMA}.SUPPLIER_CASE").fetchone()[0])

print("\nSample rows:")
for r in cs.execute(f"""
    SELECT SupplierID, SupplierName, PhoneNumber, WebsiteURL,
           TRY_TO_DATE(ValidFrom) AS ValidFrom, TRY_TO_DATE(ValidTo) AS ValidTo
    FROM {DB}.{SCHEMA}.SUPPLIER_CASE
    ORDER BY SupplierID
    LIMIT 10
""").fetchall():
    print(r)

print("\nSchema:")
for r in cs.execute(f"DESCRIBE TABLE {DB}.{SCHEMA}.SUPPLIER_CASE").fetchall():
    print(r[0], r[1])


Rows: 13

Sample rows:
(1, 'A Datum Corporation', '(847) 555-0100', 'http://www.adatum.com', None, None)
(2, 'Contoso, Ltd.', '(360) 555-0100', 'http://www.contoso.com', None, None)
(3, 'Consolidated Messenger', '(415) 555-0100', 'http://www.consolidatedmessenger.com', None, None)
(4, 'Fabrikam, Inc.', '(203) 555-0104', 'http://www.fabrikam.com', None, None)
(5, 'Graphic Design Institute', '(406) 555-0105', 'http://www.graphicdesigninstitute.com', None, None)
(6, 'Humongous Insurance', '(423) 555-0105', 'http://www.humongousinsurance.com', None, None)
(7, 'Litware, Inc.', '(209) 555-0108', 'http://www.litwareinc.com', None, None)
(8, 'Lucerne Publishing', '(423) 555-0103', 'http://www.lucernepublishing.com', None, None)
(9, 'Nod Publishers', '(252) 555-0100', 'http://www.nodpublishers.com', None, None)
(10, 'Northwind Electric Cars', '(201) 555-0105', 'http://www.northwindelectriccars.com', None, None)

Schema:
SUPPLIERID NUMBER(38,0)
SUPPLIERNAME VARCHAR(16777216)
SUPPLIERCATEGORYID N

In [13]:
cs.execute("USE DATABASE TESTDB")
cs.execute("USE SCHEMA PUBLIC")

cs.execute("""
CREATE OR REPLACE TABLE TESTDB.PUBLIC.SUPPLIER_CASE_CLEAN AS
SELECT
  CAST(SUPPLIERID               AS INT)        AS SUPPLIERID,
  SUPPLIERNAME                                   AS SUPPLIERNAME,
  CAST(SUPPLIERCATEGORYID       AS INT)        AS SUPPLIERCATEGORYID,
  CAST(PRIMARYCONTACTPERSONID   AS INT)        AS PRIMARYCONTACTPERSONID,
  CAST(ALTERNATECONTACTPERSONID AS INT)        AS ALTERNATECONTACTPERSONID,
  CAST(DELIVERYMETHODID         AS INT)        AS DELIVERYMETHODID,
  CAST(POSTALCITYID             AS INT)        AS POSTALCITYID,
  SUPPLIERREFERENCE                              AS SUPPLIERREFERENCE,
  PHONENUMBER                                   AS PHONENUMBER,
  WEBSITEURL                                    AS WEBSITEURL,
  DELIVERYADDRESSLINE1                           AS DELIVERYADDRESSLINE1,
  CAST(DELIVERYPOSTALCODE       AS INT)        AS DELIVERYPOSTALCODE,
  POSTALADDRESSLINE1                             AS POSTALADDRESSLINE1,
  CAST(POSTALPOSTALCODE         AS INT)        AS POSTALPOSTALCODE,
  CAST(LASTEDITEDBY             AS INT)        AS LASTEDITEDBY,
  TRY_TO_DATE(VALIDFROM)                        AS VALIDFROM,
  TRY_TO_DATE(VALIDTO)                          AS VALIDTO
FROM TESTDB.PUBLIC.SUPPLIER_CASE;
""")


In [14]:
print(cs.execute("SELECT COUNT(*) FROM TESTDB.PUBLIC.SUPPLIER_CASE_CLEAN").fetchone()[0])
for r in cs.execute("""
  SELECT SUPPLIERID, SUPPLIERNAME, PHONENUMBER, WEBSITEURL, VALIDFROM, VALIDTO
  FROM TESTDB.PUBLIC.SUPPLIER_CASE_CLEAN
  ORDER BY SUPPLIERID
  LIMIT 10
""").fetchall():
    print(r)


13
(1, 'A Datum Corporation', '(847) 555-0100', 'http://www.adatum.com', None, None)
(2, 'Contoso, Ltd.', '(360) 555-0100', 'http://www.contoso.com', None, None)
(3, 'Consolidated Messenger', '(415) 555-0100', 'http://www.consolidatedmessenger.com', None, None)
(4, 'Fabrikam, Inc.', '(203) 555-0104', 'http://www.fabrikam.com', None, None)
(5, 'Graphic Design Institute', '(406) 555-0105', 'http://www.graphicdesigninstitute.com', None, None)
(6, 'Humongous Insurance', '(423) 555-0105', 'http://www.humongousinsurance.com', None, None)
(7, 'Litware, Inc.', '(209) 555-0108', 'http://www.litwareinc.com', None, None)
(8, 'Lucerne Publishing', '(423) 555-0103', 'http://www.lucernepublishing.com', None, None)
(9, 'Nod Publishers', '(252) 555-0100', 'http://www.nodpublishers.com', None, None)
(10, 'Northwind Electric Cars', '(201) 555-0105', 'http://www.northwindelectriccars.com', None, None)
